In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tqdm import tqdm
from classic_network import LeNet_5, AlexNet, VGG16
%matplotlib inline

# LeNet-5

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

INPUT_SIZE = (32, 32, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2)), 'constant').reshape((y_train.size, *INPUT_SIZE))
x_test = np.pad(x_test, ((0,0),(2,2),(2,2)), 'constant').reshape((y_test.size, *INPUT_SIZE))

In [3]:
tf.reset_default_graph()

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 1])
y = tf.placeholder(tf.int64, shape=[None])
one_hot_y = tf.one_hot(y, 10)

In [5]:
LR = 0.001
BATCH_SIZE = 16
EPOCHS = 4

logits, params = LeNet_5(x)
pred = tf.argmax(logits, axis=-1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot_y,
                                                           logits=logits)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=LR)
train_op = optimizer.minimize(loss)

correct_pred = tf.equal(pred, y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) * 100

tf.summary.scalar('acc_summary', accuracy)
merged = tf.summary.merge_all()

In [7]:
# TODO Run model and evaluate
STORE_PATH = 'logs/'
saver = tf.train.Saver()
ws = {}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(STORE_PATH, sess.graph)

    for i in range(EPOCHS):

        writer = tf.summary.FileWriter(STORE_PATH, sess.graph)
        x_train, y_train = shuffle(x_train, y_train)
        for offset in tqdm(range(0, y_train.size, BATCH_SIZE)):
            end = offset + BATCH_SIZE
            xt, yt = x_train[offset:end], y_train[offset:end]
            sess.run(train_op, feed_dict={x: xt, y: yt})


        loss_v, acc, summary = sess.run([loss, accuracy, merged], feed_dict={x: xt, y: yt})
        print("Epoach {}, Minibatch Loss= {:.4f}, Training Accuracy={:.2f}%".format(i, loss_v, acc))
        writer.add_summary(summary, i)

    print("Optimization Finished!")
    save_path = saver.save(sess, "logs/lenet.ckpt")

    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: x_test,
                                      y: y_test}))
    ws = sess.run(params)

  0%|          | 0/3750 [00:00<?, ?it/s]

Epoach 0, Minibatch Loss= 0.0385, Training Accuracy=100.00%


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoach 1, Minibatch Loss= 0.0125, Training Accuracy=100.00%


  0%|          | 0/3750 [00:00<?, ?it/s]

Epoach 2, Minibatch Loss= 0.0014, Training Accuracy=100.00%


100%|██████████| 3750/3750 [01:07<00:00, 55.22it/s]


Epoach 3, Minibatch Loss= 0.0031, Training Accuracy=100.00%
Optimization Finished!
Testing Accuracy: 98.24


In [11]:
with tf.Session() as sess:
    saver.restore(sess, "logs/lenet.ckpt")
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: x_test,
                                      y: y_test}))

INFO:tensorflow:Restoring parameters from logs/lenet.ckpt
Testing Accuracy: 98.24


# AlexNet

In [9]:
tf.reset_default_graph()

In [8]:
# TODO: Download ImageNet

In [9]:
x = tf.placeholder(tf.float32, shape=[None, 227, 227, 3])
y = tf.placeholder(tf.int32, shape=[None])
one_hot_y = tf.one_hot(y, 1000)

In [10]:
BS = 128
Mom = 0.9
LR = 0.0005
logits = AlexNet(x)
cost = tf.losses.softmax_cross_entropy(one_hot_y, logits)
optimizer = tf.train.MomentumOptimizer(learning_rate=LR,
                                       momentum=Mom)
training = optimizer.minimize(cost)

# VGG16

In [11]:
tf.reset_default_graph()

In [12]:
# TODO: Crop ImageNet

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3])
y = tf.placeholder(tf.int32, shape=[None])
one_hot_y = tf.one_hot(y, 1000)

In [14]:
BS = 256
Mom = 0.9
LR = 1e-2
logits = VGG16(x)
cost = tf.losses.softmax_cross_entropy(one_hot_y, logits)
optimizer = tf.train.MomentumOptimizer(learning_rate=LR,
                                       momentum=Mom)
training = optimizer.minimize(cost)